# Merge, analyse and visualize OGGM GCM runs

In this notebook we want to show: 
- How to merge the output of different OGGM GCM projections into one dataset
- How to calculate regional values (e.g. volume) using the merged dataset
- How to use matplotlib or [seaborn](https://seaborn.pydata.org/index.html) to visualize the projections and plot different statistical estimates (median, interquartile range, mean, std, ...)
- How to use [HoloViews](https://holoviews.org/) and [Panel](https://panel.holoviz.org/) to visualize the outcome. This part uses advanced plotting capabilities which are not necessary to understand the rest of the notebook.

This notebook is intended to explain the postprocessing steps, rather than the OGGM workflow itself. Therefore some code (especially conducting the GCM projection runs) does not have many explanations. If you are more interested in these steps you should check out the notebook [Run OGGM with GCM data](run_with_gcm.ipynb).

## GCM projection runs

The first step is to conduct the GCM projection runs. We choose two different glaciers by their rgi_ids and conduct the GCM projections. Again if you do not understand all of the following code you should check out the [Run OGGM with GCM data](run_with_gcm.ipynb) notebook.

In [ ]:
# Libs
from time import gmtime, strftime
import xarray as xr
import numpy as np

# Locals
from oggm import utils, workflow, tasks
import oggm.cfg as cfg
from oggm.shop import gcm_climate

### Pre-processed directories

In [ ]:
# Initialize OGGM and set up the default run parameters
cfg.initialize(logging_level='WARNING')

# change border around the individual glaciers
cfg.PARAMS['border'] = 80

# Use Multiprocessing
cfg.PARAMS['use_multiprocessing'] = True

# For hydro output
cfg.PARAMS['store_model_geometry'] = True

# Local working directory (where OGGM will write its output)
cfg.PATHS['working_dir'] = utils.gettempdir('OGGM_merge_gcm_runs', reset=True)

# RGI glaciers: Ngojumba and Khumbu
rgi_ids = utils.get_rgi_glacier_entities(['RGI60-15.03473', 'RGI60-15.03733'])

# Go - get the pre-processed glacier directories
# in OGGM v1.6 you have to explicitly indicate the url from where you want to start from
# we will use here the elevation band flowlines which are much simpler than the centerlines
base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.1/elev_bands/W5E5/')
gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=5,
                                          prepro_base_url=base_url)

### Download and process GCM data

We prepared for you two options:
- You can either use the ISIMIP3b files that are bias-corrected CMIP6 GCMs
- or use directly CMIP5 (differences explained shortly in [run_with_gcm](run_with_gcm.ipynb)).
We will use here per default ISIMIP3b, because then you need to download less data. 

The subsequent postprocessing and plotting code works for both options. If you want to do the visualization with Holoviews, it only works with `option='CMIP5`.

In [ ]:
option = 'isimip3b_CMIP6'
# option = 'CMIP5'    ## Attention: This takes quite long and needs to download much more data 

In [ ]:
if option == 'isimip3b_CMIP6':
    all_GCM = ['gfdl-esm4_r1i1p1f1', 'ipsl-cm6a-lr_r1i1p1f1',
               'mpi-esm1-2-hr_r1i1p1f1',
               'mri-esm2-0_r1i1p1f1', 'ukesm1-0-ll_r1i1p1f2']

    # define the SSP scenarios
    all_scenario = ['ssp126', 'ssp370', 'ssp585']

    for GCM in all_GCM:
        for ssp in all_scenario:
            # Download and process them:
            workflow.execute_entity_task(gcm_climate.process_monthly_isimip_data, gdirs, 
                                         ssp = ssp,
                                         # gcm ensemble -> you can choose another one
                                         member=GCM,
                                         # recognize the climate file for later
                                         output_filesuffix=f'_{GCM}_{ssp}'
                                         );
elif option=='CMIP5':
    # define the GCM models you want to use
    all_GCM = ['CCSM4',
               'CNRM-CM5',
               'CSIRO-Mk3-6-0',
              ]

    # define the SSP scenarios to use
    all_scenario = ['rcp26',
                   #'rcp45',# let's only take three scenarios, that is faster
                   'rcp60', 
                   'rcp85']

    # download locations for precipitation and temperature
    bp = 'https://cluster.klima.uni-bremen.de/~oggm/cmip5-ng/pr/pr_mon_{}_{}_r1i1p1_g025.nc'
    bt = 'https://cluster.klima.uni-bremen.de/~oggm/cmip5-ng/tas/tas_mon_{}_{}_r1i1p1_g025.nc'

    for GCM in all_GCM:
        for rcp in all_scenario:
            # Download the files
            ft = utils.file_downloader(bt.format(GCM, rcp))
            fp = utils.file_downloader(bp.format(GCM, rcp))

            try:
                # bias correct them
                workflow.execute_entity_task(gcm_climate.process_cmip_data, gdirs, 
                                             filesuffix='_{}_{}'.format(GCM, rcp),  # recognize the climate file for later
                                             fpath_temp=ft,  # temperature projections
                                             fpath_precip=fp,  # precip projections
                                             );
            except ValueError:
                # if a certain scenario is not available for a GCM we land here
                # and we inidcate this by printing a message so the user knows
                # this scenario is missing
                print('No ' + GCM +' run with scenario ' + rcp + ' available!')

Here we defined, downloaded and processed all the GCM models and scenarios (either SSP or RCP) that we want to use. In case of CMIP5, you can get [here](https://cluster.klima.uni-bremen.de/~oggm/cmip5-ng/gcm_table.html) an overview of all available options. As you see in the overview only for two of the three GCMs for CMIP5 the rcp60 Scenario is available! We deal with this issue by including a ```try```/```except``` in the code below.

### Actual projection Runs

Now we conduct the actual projection runs. Again handling the case that for a certain (GCM, SCENARIO) combination no data is available with a ```try```/```except```.

In [ ]:
for GCM in all_GCM:
    for scen in all_scenario:
        rid = '_{}_{}'.format(GCM, scen)
        try:  # check if (GCM, scen) combination exists
            workflow.execute_entity_task(tasks.run_with_hydro, gdirs,
                                         run_task=tasks.run_from_climate_data,
                                         ys=2020,  # star year of our projection runs
                                         climate_filename='gcm_data',  # use gcm_data, not climate_historical
                                         climate_input_filesuffix=rid,  # use the chosen GCM and scenario
                                         init_model_filesuffix='_historical',  # this is important! Start from 2020 glacier
                                         output_filesuffix=rid,  # the filesuffix of the resulting file, so we can find it later
                                         store_monthly_hydro=True
                                        );

        except FileNotFoundError:
            # if a certain scenario is not available for a GCM we land here
            # and we inidcate this by printing a message so the user knows
            # this scenario is missing
            print('No ' + GCM +' run with scenario ' + scen + ' available!')

## Merge datasets together

Now that we have conducted the projection Runs we can start merging all outputs into one dataset. The individual files can be opened by providing the ```output_filesuffix``` of the projection runs as the ```input_filesuffix```. Let us have a look at one resulting dataset!

In [ ]:
file_id  = '_{}_{}'.format(all_GCM[0], all_scenario[0])
ds = utils.compile_run_output(gdirs, input_filesuffix=file_id)
ds

We see that as a result, we get a [xarray.Dataset](https://xarray.pydata.org/en/stable/generated/xarray.Dataset.html). We also see that this dataset has three dimensions ```time```, ```rgi_id``` and ```month_2d```. When we look at the ```Data variables``` (click to expand) we can see that for each of these variables a certain combination of the dimensions is given. For example, for the ```volume``` we see the dimensions ```(time, rgi_id)``` which indicates that the underlying data is separated by each year and each glacier. Let's have a look at the volume in 2030 and 2040 at the glacier 'RGI60-15.03473' (Ngojumba):

In [ ]:
ds['volume'].loc[{'time': [2030, 2040], 'rgi_id': ['RGI60-15.03473' ]}]

A more complete overview of how to access the data of xarray Dataset can be found [here](https://xarray.pydata.org/en/stable/user-guide/indexing.html#dataset-indexing).

This is quite useful, but unfortunately, we have one xarray Dataset for each (GCM, RCP) pair. For an easier calculation and comparison between different GCMs and Scenarios(SSPs or RCPs), it would be desirable to combine all individual Datasets into one Dataset with the additional dimensions ```(GCM, SCENARIO)```. Fortunately, xarray provides different functions to merge Datasets ([here](https://xarray.pydata.org/en/stable/user-guide/combining.html) you can find an Overview).

In our case, we want to merge all individual datasets with two additional coordinates ```GCM``` and ```SCENARIO```. Additional coordinates can be added with the comment ```.coords[]```, also you can include a description with ```.coords[].attrs['description']```. For example, let us add ```GCM``` as a new coordinate:

In [ ]:
ds.coords['GCM'] = all_GCM[0]
ds.coords['GCM'].attrs['description'] = 'Global Circulation Model'
ds

You can see that now ```GCM``` is added as Coordinate, but when you look at the Data variables you see that this new Coordinate is not used by any of them. So we must add this new Coordinate to the Data variables using the ```.expand_dims()``` comment:

In [ ]:
ds = ds.expand_dims('GCM')
ds

Now we see that ```GCM``` was added to the Dimensions and all Data variables now use this new coordinate. The same can be done with ```SCENARIO```. As a standalone, this is not very useful. But if we add these coordinates to all datasets, it becomes quite handy for merging. Therefore we now open all datasets and add to each one the two coordinates ```GCM``` and ```SCENARIO```:

### Add new coordinates

In [ ]:
ds_all = []  # in this array all datasets going to be stored with additional coordinates GCM and RCP
creation_date = strftime("%Y-%m-%d %H:%M:%S", gmtime())  # here add the current time for info
for GCM in all_GCM:  # loop through all GCMs
    for scen in all_scenario:  # loop through all RCPs
        try:  # check if GCM, RCP combination exists
            rid = '_{}_{}'.format(GCM, scen)  # put together the same filesuffix which was used during the projection runs

            ds_tmp = utils.compile_run_output(gdirs, input_filesuffix=rid)  # open one model run

            ds_tmp.coords['GCM'] = GCM  # add GCM as a coordinate
            ds_tmp.coords['GCM'].attrs['description'] = 'used Global circulation Model'  # add a description for GCM
            ds_tmp = ds_tmp.expand_dims("GCM")  # add GCM as a dimension to all Data variables

            ds_tmp.coords['SCENARIO'] = scen  # add scenario (ssp or rcp) as a coordinate
            ds_tmp.coords['SCENARIO'].attrs['description'] = 'used scenario (SSP or RCP)'
            ds_tmp = ds_tmp.expand_dims("SCENARIO")  # add RCP as a dimension to all Data variables

            ds_tmp.attrs['creation_date'] = creation_date  # also add todays date for info
            ds_all.append(ds_tmp)  # add the dataset with extra coordinates to our final ds_all array

        except RuntimeError as err:  # here we land if an error occured
            if str(err) == 'Found no valid glaciers!':  # This is the error message if the GCM, RCP combination does not exist
                print(f'No data for GCM {GCM} with SCENARIO {scen} found!')  # print a descriptive message
            else:
                raise RuntimeError(err)  # if an other error occured we just raise it

### Actual merging

```ds_all``` now contains all outputs with the additionally added coordinates. Now we can merge them with the very convenient xarray function ```xr.combine_by_coords()```:

In [ ]:
ds_merged = xr.combine_by_coords(ds_all, fill_value=np.nan)  # define how the missing GCM, RCP combinations should be filled

Great! You see that the result is one Dataset, but we still can identify the individual runs with the two additionally added Coordinates GCM and RCP. If you want to save this new Dataset for later analysis you can use for example ```.to_netcdf()``` ([here](https://xarray.pydata.org/en/stable/user-guide/io.html) you can find an Overview of xarray writing options):

```python
ds_merged.to_netcdf('merged_GCM_projections.nc')
```

And to open it again later and automatically close the file after reading you can use:

```python
with xr.open_dataset('merged_GCM_projections.nc') as ds:
    ds_merged = ds
```

## Calculate regional values

Now we have our merged dataset and we can start with the analysis. To do this xarray gives us a ton of different mathematical functions which can be used directly with our dataset. [Here](https://xarray.pydata.org/en/stable/user-guide/computation.html) you can find a great overview of what is possible.

As an example, we could calculate the mean and std of the total regional glacier volume for different scenarios of our merged dataset. Here our added coordinates come in very handy. However, for a small amount of GCMs (for ISIMIP3b e.g. 5 GCMs), it is often better to compute instead more robust estimates, such as the median, the 25th and 75th percentile or the total range. We will show different options later:

Let us start by calculating the total "regional" glacier volume using ```.sum()``` (of course, here is is just the sum over two glaciers...):

In [ ]:
ds_total_volume = ds_merged['volume'].sum(dim='rgi_id',  # over which dimension the sum should be taken, here we want to sum up over all glacier ids
                                          skipna=True,  # ignore nan values
                                          # important, we need values for every glacier (here 2) to do the sum
                                          # if some have NaN, the sum should also be NaN (if you don't set min_count, the sum will be 0, which is bad)
                                          min_count=len(ds_merged.rgi_id), 
                                          keep_attrs=True)  # keep the variable descriptions                                                              

For this calculation, we told xarray over which dimension we want to sum up ```dim='rgi_id'``` (all glaciers ), that missing values should be ignored ```skipna=True``` (as some GCM/RCP combinations are not available) and that the attributes (the description of the variables) should be preserved ```keep_attrs=True```. You can see that the result has three dimensions left ```(GCM, time, RCP)``` and the dimension ```rgi_id``` disappeared as we have summed all values up along this dimension.

Likewise, we can now calculate for example the median of all GCM runs:

In [ ]:
ds_total_volume_median = ds_total_volume.median(dim='GCM',  # over which dimension the median should be calculated
                                            skipna=True,  # ignore nan values
                                            keep_attrs=True)  # keep all variable descriptions
ds_total_volume_median

And you see that now we are left with two dimensions ```(SCENARIO, time)```. This means we have calculated the median total volume for all different scenarios (SSP or RCP) and along the projection period. The mean, standard deviation (std) or percentiles can be also calculated in the same way as the median. Again, bare in mind, that for small sample sizes of GCM ensembles (15), which is almost always the case, it is often much more robust to use median and the interquartile range or other percentile ranges. 

## Visualisation with matplotlib or seaborn
We will first use matplotlib, where we need to estimate the statistical estimates ourselves. This is good to understand what we are doing. 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# estimate the minimum volume for each time step and scenario over the GCMs
ds_total_volume_min = ds_total_volume.min(dim='GCM', keep_attrs=True,skipna=True)
# estimate the 25th percentile volume for each time step and scenario over the GCMs
ds_total_volume_p25 = ds_total_volume.quantile(0.25, dim='GCM', keep_attrs=True,skipna=True)
# estimate the 50th percentile volume for each time step and scenario over the GCMs
ds_total_volume_p50 = ds_total_volume.quantile(0.5, dim='GCM', keep_attrs=True,skipna=True)
# this is the same as the median -> Let's check
np.testing.assert_allclose(ds_total_volume_median,ds_total_volume_p50)
# estimate the 75th percentile volume for each time step and scenario over the GCMs
ds_total_volume_p75 = ds_total_volume.quantile(0.75, dim='GCM', keep_attrs=True,skipna=True)
# estimate the maximum volume for each time step and scenario over the GCMs
ds_total_volume_max = ds_total_volume.max(dim='GCM', keep_attrs=True,skipna=True)

# Think twice if it is appropriate to compute a mean/std over your GCM sample, is it Gaussian distributed?
# Otherwise use instead median and percentiles or total range
ds_total_volume_mean = ds_total_volume.mean(dim='GCM', keep_attrs=True,
                                            skipna=True)
ds_total_volume_std = ds_total_volume.std(dim='GCM', keep_attrs=True,
                                            skipna=True)

In [ ]:
if option == 'isimip3b_CMIP6':
    color_dict={'ssp126':'blue', 'ssp370':'orange', 'ssp585':'red'}
elif option == 'CMIP5':
    color_dict={'rcp26':'blue', 'rcp60':'violet', 'rcp85':'red'}

fig, axs = plt.subplots(1,2, figsize=(12,6), 
                        sharey=True # we want to share the y axis betweeen the subplots
                       )
for scenario in color_dict.keys():
    # get amount of GCMs per Scenario to add it to the legend:
    n = len(ds_total_volume.sel(SCENARIO=scenario).dropna(dim='GCM').GCM)
    axs[0].plot(ds_total_volume_median.time,
                ds_total_volume_median.sel(SCENARIO=scenario)/1e9,  # m3 -> km3
                label=f'{scenario}: n={n} GCMs',
                color=color_dict[scenario],lw=3)
    axs[0].fill_between(ds_total_volume_median.time,
                        ds_total_volume_p25.sel(SCENARIO=scenario)/1e9,
                        ds_total_volume_p75.sel(SCENARIO=scenario)/1e9,
                        color=color_dict[scenario],
                        alpha=0.5,
                        label='interquartile range\n(75th-25th percentile)')
    axs[0].fill_between(ds_total_volume_median.time,
                        ds_total_volume_min.sel(SCENARIO=scenario)/1e9,
                        ds_total_volume_max.sel(SCENARIO=scenario)/1e9,
                        color=color_dict[scenario],
                        alpha=0.1,
                        label='total range')


for scenario in color_dict.keys():
    axs[1].plot(ds_total_volume_mean.time,
                 ds_total_volume_mean.sel(SCENARIO=scenario)/1e9,  # m3 -> km3
                 color=color_dict[scenario],
                 label='mean',lw=3)
    axs[1].fill_between(ds_total_volume_mean.time,
                        ds_total_volume_mean.sel(SCENARIO=scenario)/1e9 - ds_total_volume_std.sel(SCENARIO=scenario)/1e9,
                        ds_total_volume_mean.sel(SCENARIO=scenario)/1e9 + ds_total_volume_std.sel(SCENARIO=scenario)/1e9,
                        alpha=0.3,
                        color=color_dict[scenario],
                       label='standard deviation')
    
for ax in axs:
    handles, labels = ax.get_legend_handles_labels()
    if ax == axs[0]:
        leg1 = ax.legend(handles[:3], labels[:3], title='Scenario')
        ax.set_ylabel(r'Volume (km$^3$)')
        ax.legend([handles[0],handles[3],handles[4]],
                  ['median',labels[3],labels[4]], loc='lower left')
        ax.add_artist(leg1)
    else:
        # we only want to have the legend for mean and std
        ax.legend(handles[::3], labels[::3], loc='lower left') 
    ax.set_xlabel('Year');
    ax.grid()

You have to choose which statistical estimate is best in your case. It is also a good possibility to just plot all the scenarios, and then to decide which statistical estimate describes best the spread in your case:

In [ ]:
# for example, if there are just 3 GCMs, maybe you can just show all of them?
for gcm in all_GCM: 
    plt.plot(ds_total_volume.sel(SCENARIO=scenario).time,
            ds_total_volume.sel(SCENARIO=scenario, GCM=gcm),
             color='grey')
plt.title(scenario)

You could create similar plots even easier with  [seaborn](https://seaborn.pydata.org/index.html), which has a lot of statistical estimation tools directly included (specifically in seaborn>=v0.12). You can for example check out [this errorbar seaborn tutorial](https://seaborn.pydata.org/tutorial/error_bars.html). Note that the outcome can be slightly different even for the same statistical estimate as e.g. seaborn and xarray might use different methods to compute the same thing. 

In [ ]:
import seaborn as sns 
# this code might only work with seaborn v>=0.12
# seaborn always likes to have pandas dataframes
pd_total_volume = ds_total_volume.to_dataframe('volume_m3').reset_index()
sns.lineplot(x='time', y='volume_m3',
             hue='SCENARIO', # these are the dimension for the different colors 
             estimator='median', # here you could also choose the mean
             data=pd_total_volume,
             # here you could also choose another percentile range
             # "90" means the range goes from the 5th to the 95th percentile
             # (50 would be the interquartile range, i.e., the same as two plots above)
             errorbar=('pi', 90) 
            );

## Interactive visualisation with HoloViews and Panel
- the following code only works if you install holoviews and panel

We can also visualize the data using tools from the [HoloViz](https://holoviz.org/) framework (namely [HoloViews](https://holoviews.org/) and [Panel](https://panel.holoviz.org/)). For an introduction to HoloViz, you can have a look at the [Small overview of HoloViz capability of data exploration](holoviz_intro.ipynb) notebook.

This part of the code only works at the moment if you set 
option=`CMIP5` when you preprocess the projection data. 

In [ ]:
# Plotting
import holoviews as hv
import panel as pn
hv.extension('bokeh')
pn.extension()

To make your life easier with Holoviews and Panel, you can define a small function that computes your estimates:
- In the following section, we only show mean and std, but this could be similar applied to the median and specific percentiles (which might be more robust in your use case :-) ). But just for showing how it works, it is fine to use here mean and std!

In [ ]:
def calculate_total_mean_and_std(ds, variable):
    mean = ds[variable].sum(dim='rgi_id',  # first sum up over all glaciers
                            skipna=True,
                            keep_attrs=True,
                            # important, need values for every glacier
                            min_count=len(ds_merged.rgi_id), 
                           ).mean(dim='GCM',  # afterwards calculate the mean of all GCMs
                                  skipna=True,
                                  keep_attrs=True,
                                 )
    std = ds[variable].sum(dim='rgi_id',  # first sum up over all glaciers 
                           skipna=True,
                           keep_attrs=True,
                           min_count=len(ds_merged.rgi_id), 
                          ).std(dim='GCM',  # afterwards calculate the std of all GCMs
                                skipna=True,
                                keep_attrs=True,
                               )
    return mean, std

This function takes a dataset ```ds``` and a variable name ```variable``` for which the mean and the std should be calculated. This function will come in handy in the next section dealing with the visualisation of the calculated values.

First, we create a single curve for a single RCP scenario:

In [ ]:
# calculate mean and std with the previously defined function
total_volume_mean, total_volume_std = calculate_total_mean_and_std(ds_merged, 'volume')

# select only on RCP scenario
total_volume_mean_ssp585 = total_volume_mean.loc[{'SCENARIO': 'ssp585'}]

# plot a curve
x = total_volume_mean_ssp585.coords['time']
y = total_volume_mean_ssp585
hv.Curve((x, y),
         kdims=x.name,
         vdims=y.name,
        ).opts(xlabel=x.attrs['description'],
               ylabel=f"{y.attrs['description']} in {y.attrs['unit']}")

We used a [HoloViews Curve](https://holoviews.org/reference/elements/bokeh/Curve.html) and defined ```kdims``` and ```vdims```. This definition is not so important for a single plot but if we start to compose different plots all axis of the different plots with the same ```kdims``` or ```vdims``` are connected. This means for example whenever you zoom in on one plot all other plots also zoom in. Further, you can see that we have defined ```xlabel``` and ```ylabel``` using the variable description of the dataset, therefore it was useful to ```keep_attrs=True``` when we calculated the total values (see above).

As a next step we can add the std as a shaded area ([HoloViews Area](https://holoviews.org/reference/elements/bokeh/Area.html)) and again define the whole plot as a single curve in a new function:

### Create single mean curve with std area

In [ ]:
def get_single_curve(mean, std, ssp):
    
    mean_use = mean.loc[{'SCENARIO': ssp}]  # read out the mean of the SSP to plot 
    std_use = std.loc[{'SCENARIO': ssp}]  # read out the std of the SSP to plot
    time = mean.coords['time']  # get the time for the x axis
    
    return (hv.Area((time,  # plot std as an area
                     mean_use + std_use,  # upper boundary of the area
                     mean_use - std_use),  # lower boundary of the area
                    vdims=[mean_use.name, 'y2'],  # vdims for both boundaries
                    kdims='time',
                    label=ssp,
                   ).opts(alpha=0.2,
                          line_color=None,
                         ) *
            hv.Curve((time, mean_use),
                     vdims=mean_use.name,
                     kdims='time',
                     label=ssp,
                    )
           ).opts(width=400,  # width of the total plot
                  height=400,  # height of the total plot
                  xlabel=time.attrs['description'],
                  ylabel=f"{mean_use.attrs['description']} in {mean_use.attrs['unit']}",
                 )

### Overlay different scenarios

The single curves of the different scenarios we can put together in a [HoloViews HoloMap](https://holoviews.org/reference/containers/bokeh/HoloMap.html), which is comparable to a dictionary. This further can be easily used to create a nice overlay of all curves.

In [ ]:
def overlay_scenarios(ds, variable):
    hmap = hv.HoloMap(kdims='Scenarios')   # create a HoloMap
    mean, std = calculate_total_mean_and_std(ds, variable)  # calculate mean and std for all SSPs using our previously defined function
    for ssp in all_scenario:
        hmap[ssp] = get_single_curve(mean, std, ssp)  # add a curve for each SSP to the HoloMap, using the RCP as a key (when you compare it do a dictonary)
    return hmap.overlay().opts(title=variable)  # create an overlay of all curves

### Show different variables in one figure and save as html file

Now that we have defined how our plots should look like we can compose different variables we want to explore in one plot. To do so we can use Panel [Column](https://panel.holoviz.org/reference/layouts/Column.html) and [Row](https://panel.holoviz.org/reference/layouts/Row.html) for customization of the plot layout:

In [ ]:
all_plots = pn.Column(pn.Row(overlay_scenarios(ds_merged, 'volume'),
                             overlay_scenarios(ds_merged, 'area'),
                            ),
                      overlay_scenarios(ds_merged, 'melt_on_glacier'))

In [ ]:
all_plots

When you start exploring the plots by dragging them around or zooming in (using the tools of the toolboxes in the upper right corner of each plot) you see that the x-axes are connected. This makes it very convenient for example to look at different periods for all variables interactively.

You also can open the plots in a new browser tab by using

```python
all_plots.show()
```

or save it as a html file for sharing

```python
plots_to_save = pn.panel(all_plots)
plots_to_save.save('GCM_runs.html', embed=True)
```

## What's next?

- return to the [OGGM documentation](https://docs.oggm.org)
- back to the [table of contents](welcome.ipynb)